In [ ]:
# pip install pandas requests beautifulsoup4

In [5]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urlparse
from datetime import date
import unicodedata

In [6]:
def clean_text(text):
    # Normalize Unicode characters
    cleaned_text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return cleaned_text

In [7]:
def get_post_body(url, headers):
  response = requests.get(url, headers=headers)

  if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    section = soup.find('section', class_='main-content')

    if section:
      post_article = section.find('article', class_='post-content')
      if post_article:
        # post_container = post_article.find('div', class_='clearfix').text
        post_container = post_article.find('div', class_='clearfix')
        
        if post_container:
          post_container_text =  clean_text(post_container.get_text(separator=" ", strip=True))

          if len(post_container_text) > 10000:
            post_container_text = len(post_container_text)
            
          return post_container_text

In [8]:
def extract_date_from_url(url):
  parsed_url =  urlparse(url)
  
  path_parts = parsed_url.path.split('/')
  
  year = path_parts[1]
  month = path_parts[2]
  day = path_parts[3]
  
  date = f'{year}-{month}-{day}'
  return date

In [9]:
def get_job_postings(url, headers):
  response = requests.get(url, headers=headers)

  if response.status_code == 200:
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.content, 'html.parser')
    section = soup.find('section', class_='main-content')

    if section:
      post_container = section.find('div', class_='col-lg-8')

      job_posts = []
      
      for post in post_container.find_all('div', class_='post-classic'):
        title_heading_element = post.find('h5')

        title = clean_text(title_heading_element.find('a').text.strip())
        link = title_heading_element.find('a').get("href")

        content = get_post_body(link, headers)
        date = extract_date_from_url(link)

        job_posts.append({
          'title': title,
          'link': link,
          'content': content,
          'date': date
        })
      return job_posts
  else:
    return []

# all_jobs = get_job_postings(pagination_date_pattern, headers)
# df = pd.DataFrame(all_jobs)
# df.to_csv('job_postings.csv', index=False)

In [ ]:
def aggregate_job_postings():
  headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
  }

  first_year = 2018
  first_month = 1

  # year_today = 2018
  # month_today = 1 

  date_today = date.today()
  year_today = date_today.year
  month_today = date_today.month
  # day_today = date_today.day

  base_url = 'https://opportunitiesforyoungkenyans.co.ke'

  jobs_from_all_dates = []
  merged_jobs = []

  for year in range(first_year, year_today + 1):
    if year == year_today:
      last_month = month_today
    else:
      last_month = 12
    
    for month in range(first_month, last_month + 1):
      current_month_url = f'{base_url}/{year}/{month}'
      current_month_url_response = requests.get(current_month_url, headers=headers)
      print(current_month_url)
      if current_month_url_response.status_code == 200:
        for page_count in range(1, 1000):
          page_url = f'{current_month_url}/page/{page_count}'
          page_url_response = requests.get(page_url, headers=headers)
          
          if page_url_response.status_code == 200:
            page_url_soup = BeautifulSoup(page_url_response.content, 'html.parser')
            page_url_body_tag = page_url_soup.body
            
            if page_url_body_tag and 'error404' in page_url_body_tag.get('class', []):
              print(f'Error 404 found on {page_url}. Stopping search.')
              break # Stop searching further pages because if current page is not found then there are no more pages for this date.
            else:
              jobs_per_date = get_job_postings(page_url, headers)
              jobs_from_all_dates.append(jobs_per_date)
              print(f'Found {page_url}')
          else:
            print(f'Failed to retrieve {page_url}. Status code: {page_url_response.status_code}')
            break  # Stop searching further pages on HTTP error
      else:
        print(f'Failed to retrieve {current_month_url}. Status code: {current_month_url_response.status_code}')
        continue  # Stop searching further pages on HTTP error

      print(f'Month {month} over.')
        
      df = pd.DataFrame(jobs_from_all_dates)
      df.to_csv(f'job_postings_{month}-{year}.csv', index=False)
        
      print(f'{month}-{year} spreadsheet exported')
    print(f'Year {year} over.')

  print(f'Scraping finished.')  

aggregate_job_postings()

https://opportunitiesforyoungkenyans.co.ke/2018/1
Found https://opportunitiesforyoungkenyans.co.ke/2018/1/page/1
Found https://opportunitiesforyoungkenyans.co.ke/2018/1/page/2
Found https://opportunitiesforyoungkenyans.co.ke/2018/1/page/3
Found https://opportunitiesforyoungkenyans.co.ke/2018/1/page/4
Found https://opportunitiesforyoungkenyans.co.ke/2018/1/page/5
Found https://opportunitiesforyoungkenyans.co.ke/2018/1/page/6
Found https://opportunitiesforyoungkenyans.co.ke/2018/1/page/7
Found https://opportunitiesforyoungkenyans.co.ke/2018/1/page/8
Found https://opportunitiesforyoungkenyans.co.ke/2018/1/page/9
Found https://opportunitiesforyoungkenyans.co.ke/2018/1/page/10
Found https://opportunitiesforyoungkenyans.co.ke/2018/1/page/11
Found https://opportunitiesforyoungkenyans.co.ke/2018/1/page/12
Found https://opportunitiesforyoungkenyans.co.ke/2018/1/page/13
Found https://opportunitiesforyoungkenyans.co.ke/2018/1/page/14
Found https://opportunitiesforyoungkenyans.co.ke/2018/1/page/15